In [139]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('.\材料報價總覽 - 001.csv')
target = pd.read_csv('.\材料報價總覽 - Target.csv')

In [140]:
print('original data :',data.shape)

data = pd.concat([data, target], 0)
data.reset_index(inplace=True, drop=True)
print('concat data :',data.shape)
data.tail()

original data : (170, 61)
concat data : (171, 61)


,報價日期,報價廠商,專案名,數量,外盒,外盒材質,外盒印刷,外盒加工,外盒單價,外盒新刀,...,配件單價,配件.1,配件材質.1,配件加工.1,配件數量.1,配件單價.1,包裝單價,外箱單價,專案單價,專案總價
166,2019-3-25,墨帥,漢堡,500.0,70x70x60,圓筒盒,4c,NaN,50.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,2.3,152.3,76150.0
167,2019-3-25,墨帥,漢堡,1000.0,70x70x60,圓筒盒,4c,NaN,44.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,2.3,121.3,121300.0
168,2019-1-14,墨帥,漢堡,500.0,100x100x80,圓筒盒,4c,NaN,52.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,13.0,2.3,151.3,75650.0
169,2019-1-14,墨帥,漢堡,1000.0,100x100x80,圓筒盒,4c,NaN,43.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10.0,2.3,115.8,115800.0
170,2019-12-19,墨帥,兵姬中文版,500.0,224x224x38,天地盒,4c,水消+局部光,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
index = data.iloc[:, :4]
index.head(3)

,報價日期,報價廠商,專案名,數量
0,2019-12-17,墨帥,MOTT,500.0
1,2019-12-05,墨帥,原子怪獸,1000.0
2,2019-12-05,墨帥,原子怪獸,1500.0


In [142]:
#擷取兩份卡片資料表
card = data.iloc[:, 13:21]
card2 = data.iloc[:, 21:29]
card = pd.concat([index, card], 1)
card2 = pd.concat([index, card2], 1)
card2.columns = card.columns

In [143]:
#篩選有效資料
card = card[card['卡片'].notnull()]
card2 = card2[card2['卡片'].notnull()]

#合併兩組card資料表
card = pd.concat([card, card2], 0)
card.reset_index(inplace=True, drop=True)

'''
【特別注意】
card2['卡片數量']都是整數，dtype為int64，與card合併後Object內容物type不變造成split運算錯誤。(.str只能針對str物件)
合併到card的target項目同理。
這部分必須非常細心。

↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓處理↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓↓
'''
card['卡片數量'] = card['卡片數量'].astype('str')

In [144]:
pd.DataFrame(card.isnull().sum(), columns=['Miss_Data']).T

,報價日期,報價廠商,專案名,數量,卡片,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片數量,卡片單價,卡片新刀
Miss_Data,0,0,0,0,0,0,0,1,0,0,2,139


In [145]:
#拋棄非目標的未加工道林紙
card = card.drop((card[card['卡片加工'].isnull()].index[0]))

In [146]:
pd.DataFrame(card.isnull().sum(), columns=['Miss_Data']).T

,報價日期,報價廠商,專案名,數量,卡片,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片數量,卡片單價,卡片新刀
Miss_Data,0,0,0,0,0,0,0,0,0,0,2,138


In [147]:
# card['卡片新刀'].unique()
#整理新刀欄位

label = {np.nan:0, '有':1, '特製刀膜':1, '擋板':1}
card['卡片新刀'] = card['卡片新刀'].map(label)

In [148]:
#拆解卡片尺寸

card_in = card['卡片'].str.split('\n', expand=True)
card_in.columns=['第1尺寸', '第2尺寸', '第3尺寸']
card_in = card_in.fillna('0x0')

for i, x in enumerate(card_in.columns):
    card_in['第'+str(i+1)+'尺寸面積'] = card_in[x].str.split('x', expand=True)[0].astype('float64')\
                                    * card_in[x].str.split('x', expand=True)[1].astype('float64')
card_in.head()

,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積
0,63x88,0x0,0x0,5544.0,0.0,0.0
1,63x88,0x0,0x0,5544.0,0.0,0.0
2,63x88,0x0,0x0,5544.0,0.0,0.0
3,63x88,0x0,0x0,5544.0,0.0,0.0
4,63x88,0x0,0x0,5544.0,0.0,0.0


In [149]:
#拆解卡片數量

card_inn = card['卡片數量'].str.split('\n', expand=True)
card_inn.columns=['第1數量', '第2數量', '第3數量']
card_inn = card_inn.fillna(0).astype('float64')
card_inn.head()

,第1數量,第2數量,第3數量
0,40.0,0.0,0.0
1,80.0,0.0,0.0
2,80.0,0.0,0.0
3,80.0,0.0,0.0
4,80.0,0.0,0.0


In [150]:
card_inn['總使用面積'] = card_in['第1尺寸面積'] * card_inn['第1數量'] + \
                    card_in['第2尺寸面積'] * card_inn['第2數量'] + \
                    card_in['第3尺寸面積'] * card_inn['第3數量']
card_inn.head()

,第1數量,第2數量,第3數量,總使用面積
0,40.0,0.0,0.0,221760.0
1,80.0,0.0,0.0,443520.0
2,80.0,0.0,0.0,443520.0
3,80.0,0.0,0.0,443520.0
4,80.0,0.0,0.0,443520.0


In [151]:
#合併拆解後資料

card = pd.concat([card, card_in], 1)
card = pd.concat([card, card_inn], 1)
del card_in, card_inn
card = card.drop(columns=['報價廠商', '卡片', '卡片數量'])
card.head()

,報價日期,專案名,數量,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積
0,2019-12-17,MOTT,500.0,350g中華寧波,4c/4c,撲克油+壓紋,1_收縮,58.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,40.0,0.0,0.0,221760.0
1,2019-12-05,原子怪獸,1000.0,310g日本銅西,4c/4c,撲克油+壓紋,2_收縮,39.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
2,2019-12-05,原子怪獸,1500.0,310g日本銅西,4c/4c,撲克油+壓紋,2_收縮,37.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
3,2019-12-05,原子怪獸,2000.0,310g日本銅西,4c/4c,撲克油+壓紋,2_收縮,35.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0
4,2019-12-05,原子怪獸,4000.0,310g日本銅西,4c/4c,撲克油+壓紋,2_收縮,29.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0


---

In [152]:
card[card['卡片材質'].str.split('g', expand=True)[1].isnull()]

,報價日期,專案名,數量,卡片材質,卡片印刷,卡片加工,卡片收縮,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積
103,2019-8-19,Pictell二版,2000.0,PET塑膠卡,UV,防刮處理,1_收縮,31.200,0,63x88,0x0,0x0,5544.0,0.0,0.0,60.0,0.0,0.0,332640.0
104,2019-8-19,Pictell二版,3000.0,PET塑膠卡,UV,防刮處理,1_收縮,28.972,0,63x88,0x0,0x0,5544.0,0.0,0.0,60.0,0.0,0.0,332640.0


In [153]:
card = card.drop([103,104]) #拋棄塑膠卡片

In [154]:
#拆解卡片材質

card_material = card['卡片材質'].str.split('g', expand=True)
card_material.columns=['卡片磅數', '卡片用紙']
card_material['卡片磅數'] = card_material['卡片磅數'].astype('int64')
card_material.head()

,卡片磅數,卡片用紙
0,350,中華寧波
1,310,日本銅西
2,310,日本銅西
3,310,日本銅西
4,310,日本銅西


In [155]:
card_material['卡片用紙'].unique()

array(['中華寧波', '日本銅西', '櫻花卡', '銅板紙', '銅西卡'], dtype=object)

In [156]:
card = pd.concat([card, card_material], 1)
card = card.drop(columns=['卡片材質'])
card.head()

,報價日期,專案名,數量,卡片印刷,卡片加工,卡片收縮,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積,卡片磅數,卡片用紙
0,2019-12-17,MOTT,500.0,4c/4c,撲克油+壓紋,1_收縮,58.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,40.0,0.0,0.0,221760.0,350,中華寧波
1,2019-12-05,原子怪獸,1000.0,4c/4c,撲克油+壓紋,2_收縮,39.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0,310,日本銅西
2,2019-12-05,原子怪獸,1500.0,4c/4c,撲克油+壓紋,2_收縮,37.0,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0,310,日本銅西
3,2019-12-05,原子怪獸,2000.0,4c/4c,撲克油+壓紋,2_收縮,35.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0,310,日本銅西
4,2019-12-05,原子怪獸,4000.0,4c/4c,撲克油+壓紋,2_收縮,29.2,0,63x88,0x0,0x0,5544.0,0.0,0.0,80.0,0.0,0.0,443520.0,310,日本銅西


In [157]:
#拆解卡片加工

card_mach = card['卡片加工'].str.split('+', expand=True)
card_mach.columns=['第1加工', '第2加工', '第3加工']
card_mach = card_mach.fillna('無')
card_mach.head()

,第1加工,第2加工,第3加工
0,撲克油,壓紋,無
1,撲克油,壓紋,無
2,撲克油,壓紋,無
3,撲克油,壓紋,無
4,撲克油,壓紋,無


In [158]:
card = pd.concat([card, card_mach], 1)
card = card.drop(columns=['卡片加工'])
card.head()

,報價日期,專案名,數量,卡片印刷,卡片收縮,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,...,第3尺寸面積,第1數量,第2數量,第3數量,總使用面積,卡片磅數,卡片用紙,第1加工,第2加工,第3加工
0,2019-12-17,MOTT,500.0,4c/4c,1_收縮,58.0,0,63x88,0x0,0x0,...,0.0,40.0,0.0,0.0,221760.0,350,中華寧波,撲克油,壓紋,無
1,2019-12-05,原子怪獸,1000.0,4c/4c,2_收縮,39.0,0,63x88,0x0,0x0,...,0.0,80.0,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無
2,2019-12-05,原子怪獸,1500.0,4c/4c,2_收縮,37.0,0,63x88,0x0,0x0,...,0.0,80.0,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無
3,2019-12-05,原子怪獸,2000.0,4c/4c,2_收縮,35.2,0,63x88,0x0,0x0,...,0.0,80.0,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無
4,2019-12-05,原子怪獸,4000.0,4c/4c,2_收縮,29.2,0,63x88,0x0,0x0,...,0.0,80.0,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無


In [159]:
#拆解卡片收縮

card_end = card['卡片收縮'].str.split('_', expand=True)
card_end.columns=['分牌數', '收縮方式']
card_end['分牌數'] = card_end['分牌數'].astype('int64')
card_end.head()

,分牌數,收縮方式
0,1,收縮
1,2,收縮
2,2,收縮
3,2,收縮
4,2,收縮


In [160]:
card = pd.concat([card, card_end], 1)
card = card.drop(columns=['卡片收縮'])
card.head()

,報價日期,專案名,數量,卡片印刷,卡片單價,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,...,第2數量,第3數量,總使用面積,卡片磅數,卡片用紙,第1加工,第2加工,第3加工,分牌數,收縮方式
0,2019-12-17,MOTT,500.0,4c/4c,58.0,0,63x88,0x0,0x0,5544.0,...,0.0,0.0,221760.0,350,中華寧波,撲克油,壓紋,無,1,收縮
1,2019-12-05,原子怪獸,1000.0,4c/4c,39.0,0,63x88,0x0,0x0,5544.0,...,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無,2,收縮
2,2019-12-05,原子怪獸,1500.0,4c/4c,37.0,0,63x88,0x0,0x0,5544.0,...,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無,2,收縮
3,2019-12-05,原子怪獸,2000.0,4c/4c,35.2,0,63x88,0x0,0x0,5544.0,...,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無,2,收縮
4,2019-12-05,原子怪獸,4000.0,4c/4c,29.2,0,63x88,0x0,0x0,5544.0,...,0.0,0.0,443520.0,310,日本銅西,撲克油,壓紋,無,2,收縮


----

In [162]:
#拆解日期

import datetime
card['報價日期'] = card['報價日期'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
card['報價年份'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%Y')).astype('int64')
card['報價月份'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%m')).astype('int64')
card['報價日'] = card['報價日期'].apply(lambda x: datetime.datetime.strftime(x, '%d')).astype('int64')

card = card.drop(columns=['報價日期'])

In [163]:
card.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148 entries, 0 to 150
Data columns (total 25 columns):
專案名       148 non-null object
數量        148 non-null float64
卡片印刷      148 non-null object
卡片單價      146 non-null float64
卡片新刀      148 non-null int64
第1尺寸      148 non-null object
第2尺寸      148 non-null object
第3尺寸      148 non-null object
第1尺寸面積    148 non-null float64
第2尺寸面積    148 non-null float64
第3尺寸面積    148 non-null float64
第1數量      148 non-null float64
第2數量      148 non-null float64
第3數量      148 non-null float64
總使用面積     148 non-null float64
卡片磅數      148 non-null int64
卡片用紙      148 non-null object
第1加工      148 non-null object
第2加工      148 non-null object
第3加工      148 non-null object
分牌數       148 non-null int64
收縮方式      148 non-null object
報價年份      148 non-null int64
報價月份      148 non-null int64
報價日       148 non-null int64
dtypes: float64(9), int64(6), object(10)
memory usage: 30.1+ KB


In [164]:
#Label encoding

label = ['專案名', '卡片印刷', '第1尺寸', '第2尺寸', '第3尺寸', '卡片用紙', '第1加工', '第2加工', '第3加工', '收縮方式']
for x in label:
    card[x] = card[x].astype('category').cat.codes

In [165]:
card.columns

Index(['專案名', '數量', '卡片印刷', '卡片單價', '卡片新刀', '第1尺寸', '第2尺寸', '第3尺寸', '第1尺寸面積',
       '第2尺寸面積', '第3尺寸面積', '第1數量', '第2數量', '第3數量', '總使用面積', '卡片磅數', '卡片用紙',
       '第1加工', '第2加工', '第3加工', '分牌數', '收縮方式', '報價年份', '報價月份', '報價日'],
      dtype='object')

In [166]:
'''
本次執行還在同年份不使用年份欄位
'''
card = card[['專案名', '數量', '卡片印刷', '卡片新刀', '第1尺寸', '第2尺寸', '第3尺寸', '第1尺寸面積',
       '第2尺寸面積', '第3尺寸面積', '第1數量', '第2數量', '第3數量', '總使用面積', '卡片磅數', '卡片用紙',
       '第1加工', '第2加工', '第3加工', '分牌數', '收縮方式', '報價年份', '報價月份', '報價日', '卡片單價']]
card.head()

,專案名,數量,卡片印刷,卡片新刀,第1尺寸,第2尺寸,第3尺寸,第1尺寸面積,第2尺寸面積,第3尺寸面積,...,卡片用紙,第1加工,第2加工,第3加工,分牌數,收縮方式,報價年份,報價月份,報價日,卡片單價
0,5,500.0,0,0,14,0,0,5544.0,0.0,0.0,...,0,1,3,2,1,2,2019,12,17,58.0
1,18,1000.0,0,0,14,0,0,5544.0,0.0,0.0,...,1,1,3,2,2,2,2019,12,5,39.0
2,18,1500.0,0,0,14,0,0,5544.0,0.0,0.0,...,1,1,3,2,2,2,2019,12,5,37.0
3,18,2000.0,0,0,14,0,0,5544.0,0.0,0.0,...,1,1,3,2,2,2,2019,12,5,35.2
4,18,4000.0,0,0,14,0,0,5544.0,0.0,0.0,...,1,1,3,2,2,2,2019,12,5,29.2


# 預測

In [167]:
'''
改動 預測目標
'''
target = card[card['卡片單價'].isnull()]
card = card.dropna()

In [168]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV

### LinearRegression

In [169]:
lModel = LinearRegression()
lModel.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(lModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

-27.389241277032546

In [170]:
submit_l = lModel.predict(target.iloc[:, :-1])
submit_l

array([103.0600939, 188.5163842])

### RandomForestRegression

In [171]:
rfModel = RandomForestRegressor()
rfModel.fit(card.iloc[:, : -1], card.iloc[:, -1])
cross_val_score(rfModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.6314073716929405

In [172]:
submit_rf = rfModel.predict(target.iloc[:, :-1])
submit_rf

array([105.121,  86.7  ])

### GradientBoostingRegression

In [173]:
gbModel = GradientBoostingRegressor()
gbModel.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.7012995490132474

In [174]:
submit_gb = gbModel.predict(target.iloc[:, :-1])
submit_gb

array([115.91979686,  80.100743  ])

### XGB

In [175]:
xgb = XGBRegressor()
xgb.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

[03:18:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:16] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.6896767390351968

In [176]:
submit_xgb = xgb.predict(target.iloc[:, :-1])
submit_xgb

array([119.3022  ,  63.868008], dtype=float32)

### LightGBM

In [177]:
lgbm = LGBMRegressor(objective='regression')
lgbm.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(lgbm, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

-1.7295926725412727

In [178]:
submit_lgbm = lgbm.predict(target.iloc[:, :-1])
submit_lgbm

array([109.36571288,  80.78933889])

# Submit

In [179]:
submit = []
submit.append(submit_rf)
submit.append(submit_gb)
submit.append(submit_xgb)
submit.append(submit_lgbm)
pd.DataFrame(submit, index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel']), columns=['卡片1預測', '卡片2預測'])

,卡片1預測,卡片2預測
rfModel,105.121000,86.700000
gbModel,115.919797,80.100743
xgbModel,119.302200,63.868008
lgbmModel,109.365713,80.789339


----
# Hyperparameter

In [89]:
param_grid = {'learning_rate':[0.1, 0.3, 0.5],
              'n_estimators':[100, 200, 300],
              'subsample':[1.0, 0.5, 0.3, 0.1],
              'min_samples_split':[2, 5, 10],
              'min_samples_leaf':[1, 2, 4],
              'max_depth':[3, 6, 8],
              'tol':[0.01, 0.05, 0.1]}
gridsearch = GridSearchCV(gbModel, param_grid, n_jobs=-1, verbose=1)
grid_result = gridsearch.fit(card.iloc[:, :-1], card.iloc[:, -1])

print('Best Accuracy : {}\nUsing {}'.format(grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 2916 candidates, totalling 8748 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 321 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done 1143 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 2543 tasks      | elapsed:   30.9s
[Parallel(n_jobs=-1)]: Done 4343 tasks      | elapsed:   51.7s
[Parallel(n_jobs=-1)]: Done 6543 tasks      | elapsed:  1.3min


Best Accuracy : 0.8600160496767852
Using {'learning_rate': 0.3, 'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100, 'subsample': 0.5, 'tol': 0.01}


[Parallel(n_jobs=-1)]: Done 8748 out of 8748 | elapsed:  1.7min finished
C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [90]:
hyper_gbModel = GradientBoostingRegressor(learning_rate=0.3,
                                          max_depth=6,
                                          min_samples_leaf=1,
                                          min_samples_split=2,
                                          n_estimators=100,
                                          subsample=0.5, tol=0.01)
hyper_gbModel.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(hyper_gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

0.6204224007012312

In [91]:
submit_hyper_gbModel = hyper_gbModel.predict(target.iloc[:, :-1])
submit_hyper_gbModel

array([96.66001029, 78.12271347])

In [92]:
param_grid = {'n_estimators':[100, 200, 300],
              'min_child_weight':[1, 5, 10],
             'gamma':[0.5, 1, 1.5, 2, 5],
             'subsample':[0.6, 0.8, 1.0],
             'max_depth':[3, 4, 5]}
gridsearch = GridSearchCV(xgb, param_grid, cv=5, n_jobs=-1, verbose=1)
grid_result = gridsearch.fit(card.iloc[:, :-1], card.iloc[:, -1])
print('Best Accuracy : {}\nUsing {}'.format(grid_result.best_score_, grid_result.best_params_) )

Fitting 5 folds for each of 405 candidates, totalling 2025 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  88 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 688 tasks      | elapsed:   11.1s
[Parallel(n_jobs=-1)]: Done 1688 tasks      | elapsed:   26.7s


[02:32:48] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Best Accuracy : 0.7442460565050657
Using {'gamma': 2, 'max_depth': 4, 'min_child_weight': 1, 'n_estimators': 300, 'subsample': 0.6}


[Parallel(n_jobs=-1)]: Done 2025 out of 2025 | elapsed:   33.0s finished
C:\Users\camer\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [180]:
hyper_xgb = XGBRegressor(gamma=2,
                         max_depth=4,
                         min_child_weight=1,
                         n_estimators=300,
                         subsample=0.6)
hyper_xgb.fit(card.iloc[:, :-1], card.iloc[:, -1])
cross_val_score(hyper_xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()

[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:18:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


0.7401978178714952

In [181]:
submit_hyper_xgb = hyper_xgb.predict(target.iloc[:, :-1])
submit_hyper_xgb

array([122.5753 ,  69.98325], dtype=float32)

In [182]:
submit.append(submit_hyper_gbModel)
submit.append(submit_hyper_xgb)
pd.DataFrame(submit, index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel', 'hyper_gbModel', 'hyper_xgbModel']),
             columns=['卡片1預測', '卡片2預測'])

,卡片1預測,卡片2預測
rfModel,105.121000,86.700000
gbModel,115.919797,80.100743
xgbModel,119.302200,63.868008
lgbmModel,109.365713,80.789339
hyper_gbModel,96.660010,78.122713
hyper_xgbModel,122.575302,69.983253


In [96]:
#追加信心水準
level = [cross_val_score(rfModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(lgbm, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(hyper_gbModel, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean(),
         cross_val_score(hyper_xgb, card.iloc[:, :-1], card.iloc[:, -1], cv=5).mean()]

df_level = pd.DataFrame(level, columns=['預測信心'])
df = pd.DataFrame(submit, columns=['卡片1預測', '卡片2預測'])
df = pd.concat([df, df_level], 1)
df.index=pd.Index(['rfModel', 'gbModel', 'xgbModel', 'lgbmModel', 'hyper_gbModel', 'hyper_xgbModel'])

[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:33:25] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [97]:
df

,卡片1預測,卡片2預測,預測信心
rfModel,99.382000,47.900000,0.611553
gbModel,115.323785,76.708636,0.695803
xgbModel,117.957924,63.851768,0.680779
lgbmModel,107.213047,77.947626,-1.686052
hyper_gbModel,96.660010,78.122713,0.735072
hyper_xgbModel,120.485641,71.476685,0.745152
